**Reading Dataset**

In [ ]:
import pandas as pd

data_path = "F:/courses/mlds_nactar/dataset/iris.csv"
df = pd.read_csv(data_path)
#df.head()

# Map Classes
classes = {
    'setosa': 0,
    'versicolor': 1,
    'virginica': 2
}
df['species'] = df['species'].map(classes)
#df.tail()
#print(df['species'].values)

`Additional Link`:
* [MLXtend](http://rasbt.github.io/mlxtend/)

**Split Dataset**

In [14]:


y = df['species'].values
X = df.iloc[:, 1:5].values

`processing a large data file iteratively`

In [ ]:
in_csv = "large_file.csv"
chunk_size = 100000     #[number of lines to process at each iteration]

# specify the columns that should be read
columns = ['column0', 'column1', 'column3', 'column7', 'column9',]

# get the number of lines in csv file
nlines = subprocess.check_output(['wc', '-l', in_csv])
nlines = int(nlines.split()[0])

# iteratively read csv and dump lines into the SQLite table
for i in range(0, nlines, chunk_size):  # change 0 --> 1 if the dataset contains a column header
    df = pd.read_csv(in_csv,
                     header = None,     # no header, define column header later manually
                     nrows = chunk_size,    # number of rows to read at each iteration
                     skiprows = i   # skip rows that are already read
    )

`Modin`: data handler like pandas, actually a customization of pandas, sent data in chunk rather than as a whole in pandas. [Github](https://github.com/modin-project/modin)
* Installation: `pip install modin`

In [ ]:
import modin.pandas as pd
import numpy as np

frame_data = np.random.randint(0, 100, size=(2**10, 2**8))
df = pd.DataFrame(frame_data)

`Dask`: [Github](https://github.com/dask/dask)

In [ ]:
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')

In [ ]:
import dask
import dask.dataframe as dd

df = dask.datasets.timeseries()

**KNN Classifier**

In [17]:
class KNNClassifier(object):
    import numpy as np
    def __init__(self, k, dist_fn=None):
        self.k = k
        if dist_fn is None: self.dist_fn = _euclidean_dist()
    
    def _euclidean_dist(self, a, b):
        dist = 0
        for ele_i, ele_j in zip(a,b): dist += ((ele_i - ele_j)**2)
        dist = dist**0.5
        return dist
    
    def _find_nearest(self, x):
        dist_idx_pairs = []
        for j in range(self.dataset_.shape[0]):
            d = self.dist_fn(x, self.dataset_[j])
            dist_idx_pairs.append((d,j))
        sorted_dist_idx_pairs = sorted(dist_idx_pairs)
        return sorted_dist_idx_pairs
    
    def fit(self, X, y):
        self.dataset_ = X.copy()
        self.labels_ = y.copy()
        self.possible_labels_ = np.unique(y)
        # the tailing underscore is a scikit-learn convention, means- thay are only available for this functions only.
    
    def predict(self, X):
        predictions = np.zeros(X.shape[0], dtype=int)
        for i in range(X.shape[0]):
            k_nearest = self._find_nearest(X[i])[:self.k]
            indices = [entry[1] for entry in k_nearest]
            k_labels = self.labels_[indices]
            counts = np.bincount(k_labels, minlength=self.possible_labels_.shape[0])
            pred_label = np.argmax(counts)
            predictions[i] = pred_label
            return predictions

In [ ]:
print(f"Training ...")
knn_model = KNNClassifier(k=3)
knn_model.fit(X_train, y_train)
print(f"Training Done")
#print(knn_model.predict(X_valid))